# RF-DETR to ONNX Conversion - Google Colab

Questo notebook ti guida nella conversione di modelli RF-DETR trainati (formato `.pt` o `.pth`) in formato ONNX per inferenza locale o deployment.

## 📋 Cosa faremo:

1. Installare le dipendenze necessarie
2. Caricare il tuo modello `.pt` trainato su Roboflow
3. Rilevare automaticamente la dimensione del modello
4. Convertire in formato ONNX
5. Verificare e semplificare il modello
6. Scaricare il risultato

## ⚠️ Nota Importante: PyTorch 2.9+

PyTorch 2.9+ usa `torch.export` per default, che **non è compatibile** con RF-DETR. Questo notebook usa il legacy exporter (`dynamo=False`) per compatibilità.

---

## 🔧 Step 1: Setup & Installazione Dipendenze

Installiamo tutte le librerie necessarie. Questo potrebbe richiedere 2-3 minuti.

In [ ]:
# Installa dipendenze
!pip install -q rfdetr==1.3.0 onnx==1.19.0 onnxsim==0.4.36 torch==2.8.0 torchvision

print("✅ Installazione completata!")

## 📤 Step 2: Upload del Modello .pt

Carica il tuo file `.pt` o `.pth` trainato su Roboflow.

**Opzione A**: Upload manuale (clicca il pulsante qui sotto)  
**Opzione B**: Mount Google Drive (se il file è nel tuo Drive)

In [ ]:
# OPZIONE A: Upload manuale
from google.colab import files

print("📁 Carica il tuo file .pt o .pth:")
uploaded = files.upload()

# Ottieni il nome del file caricato
checkpoint_path = list(uploaded.keys())[0]
print(f"\n✅ File caricato: {checkpoint_path}")

In [ ]:
# OPZIONE B: Mount Google Drive (esegui solo se usi Drive)
# Uncomment le righe sotto per usare Google Drive

# from google.colab import drive
# drive.mount('/content/drive')

# # Specifica il percorso del tuo file nel Drive
# checkpoint_path = '/content/drive/MyDrive/path/to/your/model.pt'
# print(f"✅ Usando file da Drive: {checkpoint_path}")

## 🔍 Step 3: Ispezione del Modello

Analizziamo il checkpoint per determinare automaticamente:
- Risoluzione (resolution)
- Hidden dimension
- Tipo di modello (Nano/Small/Medium/Base/Large/Seg)

In [ ]:
import torch
from pathlib import Path

# Carica checkpoint per ispezione
print(f"🔍 Analizzando checkpoint: {checkpoint_path}...\n")
checkpoint = torch.load(checkpoint_path, weights_only=False)
args = checkpoint.get('args', None)

# Determina tipo di modello
model_type = None
resolution = None
hidden_dim = None

if args and hasattr(args, 'resolution'):
    resolution = args.resolution
    hidden_dim = getattr(args, 'hidden_dim', None)
    patch_size = getattr(args, 'patch_size', 'N/A')
    
    print(f"📊 Informazioni modello:")
    print(f"   Resolution: {resolution}")
    print(f"   Hidden dim: {hidden_dim}")
    print(f"   Patch size: {patch_size}\n")
    
    # Determina model type
    if resolution == 384:
        model_type = 'RFDETRNano'
    elif resolution == 512:
        model_type = 'RFDETRSmall'
    elif resolution == 576:
        model_type = 'RFDETRMedium'
    elif resolution == 560:
        if hidden_dim == 256:
            model_type = 'RFDETRBase'
        elif hidden_dim == 384:
            model_type = 'RFDETRLarge'
        else:
            model_type = 'Unknown'
    else:
        model_type = 'Unknown'
        
elif 'model' in checkpoint:
    # Instance segmentation model
    if len(checkpoint['model']) == 544:
        model_type = 'RFDETRSegPreview'
        resolution = 560  # Default per Seg
    else:
        model_type = 'Unknown'
else:
    print("⚠️ Impossibile determinare automaticamente il tipo di modello")
    model_type = 'Unknown'

print(f"🎯 Tipo modello rilevato: {model_type}")
print(f"   Resolution da usare: {resolution}\n")

if model_type == 'Unknown':
    print("⚠️ ATTENZIONE: Tipo modello sconosciuto!")
    print("   Dovrai specificare manualmente MODEL_CLASS e RESOLUTION nella cella successiva.")

## 🔄 Step 4: Conversione in ONNX

Procediamo con la conversione automatica. Se il tipo di modello non è stato rilevato, modifica manualmente `MODEL_CLASS` e `RESOLUTION`.

In [ ]:
import os
import torch
import torch.onnx
import onnx
from onnxsim import simplify
from rfdetr.detr import (
    RFDETRSmall, 
    RFDETRBase, 
    RFDETRMedium, 
    RFDETRLarge,
    RFDETRNano,
    RFDETRSegPreview
)
import hashlib

# ============================================
# CONFIGURAZIONE
# ============================================

# Output path
output_path = Path(checkpoint_path).with_suffix('.onnx')

# Model class mapping
MODEL_CLASSES = {
    'RFDETRNano': RFDETRNano,
    'RFDETRSmall': RFDETRSmall,
    'RFDETRMedium': RFDETRMedium,
    'RFDETRBase': RFDETRBase,
    'RFDETRLarge': RFDETRLarge,
    'RFDETRSegPreview': RFDETRSegPreview
}

# Se rilevamento automatico fallito, modifica qui:
# model_type = 'RFDETRSmall'  # Uncomment e modifica se necessario
# resolution = 512             # Uncomment e modifica se necessario

if model_type not in MODEL_CLASSES:
    raise ValueError(f"Model type '{model_type}' non riconosciuto. Modifica manualmente nella cella.")

MODEL_CLASS = MODEL_CLASSES[model_type]

# ============================================
# CONVERSIONE
# ============================================

print(f"🚀 Iniziando conversione...")
print(f"   Input:  {checkpoint_path}")
print(f"   Output: {output_path}")
print(f"   Model:  {model_type}")
print(f"   Res:    {resolution}\n")

# Forza CPU e fallback MPS per Mac compatibility
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Carica modello
print(f"📦 Caricando modello {model_type}...")
model = MODEL_CLASS(pretrain_weights=checkpoint_path)

# Prepara il modello per export
inner_model = model.model.model
inner_model = inner_model.cpu()
inner_model.eval()
inner_model.export()  # IMPORTANTE: Fixa positional embeddings

# Crea dummy input
dummy_input = torch.randn(1, 3, resolution, resolution)

# Export a ONNX
print(f"\n⚙️ Esportando a ONNX...")
torch.onnx.export(
    inner_model,
    dummy_input,
    str(output_path),
    export_params=True,
    opset_version=17,
    do_constant_folding=True,
    input_names=['input'],
    output_names=['boxes', 'scores'],
    dynamo=False  # CRITICO: Usa legacy exporter (PyTorch 2.9+ compatibility)
)

print(f"✅ Export completato!")

# Verifica ONNX
print(f"\n🔍 Verificando modello ONNX...")
onnx_model = onnx.load(str(output_path))
onnx.checker.check_model(onnx_model)
print(f"✅ Modello valido!")

# Semplifica
print(f"\n🔧 Semplificando modello...")
model_simplified, check = simplify(onnx_model)
if check:
    onnx.save(model_simplified, str(output_path))
    print(f"✅ Modello semplificato!")
else:
    print(f"⚠️ Semplificazione fallita, usando modello non semplificato")

# Calcola checksum
print(f"\n🔐 Calcolando checksum...")
with open(output_path, 'rb') as f:
    checksum = hashlib.sha256(f.read()).hexdigest()

file_size = os.path.getsize(output_path)

# Report finale
print(f"\n{'='*60}")
print(f"✅ CONVERSIONE COMPLETATA!")
print(f"{'='*60}")
print(f"File:     {output_path}")
print(f"Size:     {file_size / (1024*1024):.1f} MB")
print(f"SHA256:   {checksum}")
print(f"Model:    {model_type}")
print(f"Res:      {resolution}x{resolution}")
print(f"{'='*60}\n")

## 📊 Informazioni sul Modello ONNX

### Input/Output Format:

**Input**: `[batch, 3, resolution, resolution]`
- Immagine RGB normalizzata
- Batch size tipicamente = 1
- Resolution dipende dal modello (384/512/560/576)

**Output boxes**: `[batch, 300, 4]`
- Bounding boxes (x1, y1, x2, y2)
- Coordinate normalizzate 0-1

**Output scores**: `[batch, 300, num_classes]`
- Score per ogni classe
- Richiede softmax per ottenere probabilità

### Post-Processing Necessario:

1. **Softmax** sugli scores
2. **Argmax** per classe predetta
3. **Confidence threshold** (es. 0.5)
4. **NMS** (Non-Maximum Suppression) per overlap

## 🧪 Step 5: Test del Modello ONNX (Opzionale)

Testiamo il modello ONNX convertito con una predizione di prova.

In [ ]:
import onnxruntime as ort
import numpy as np

print("🧪 Testing ONNX model...\n")

# Carica ONNX session
session = ort.InferenceSession(str(output_path))

# Get input/output info
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape
output_names = [out.name for out in session.get_outputs()]

print(f"📥 Input:")
print(f"   Name:  {input_name}")
print(f"   Shape: {input_shape}\n")

print(f"📤 Outputs:")
for i, out in enumerate(session.get_outputs()):
    print(f"   [{i}] {out.name}: {out.shape}")

# Crea dummy input per test
dummy_input = np.random.randn(1, 3, resolution, resolution).astype(np.float32)

# Run inference
print(f"\n🔮 Running test inference...")
outputs = session.run(None, {input_name: dummy_input})

print(f"✅ Inference completata!\n")
print(f"   Boxes shape:  {outputs[0].shape}")
print(f"   Scores shape: {outputs[1].shape}")

print(f"\n✅ Modello ONNX funzionante!")

## 💾 Step 6: Download del Modello ONNX

Scarica il file `.onnx` convertito sul tuo computer.

In [ ]:
from google.colab import files

print(f"📥 Downloading {output_path}...\n")
files.download(str(output_path))
print(f"✅ Download completato!")

## 🎯 Next Steps

### Upload su RaceTagger (Supabase)

1. Vai al **Management Portal** → **Model Manager**
2. Seleziona la categoria sport
3. Inserisci versione e note
4. Upload del file `.onnx`
5. Incolla le classi dal training Roboflow
6. Il checksum SHA256 viene verificato automaticamente

### Esempio Post-Processing Code

```python
import numpy as np

def postprocess(boxes, scores, conf_threshold=0.5, iou_threshold=0.5):
    # Softmax
    probs = np.exp(scores) / np.exp(scores).sum(axis=-1, keepdims=True)
    
    # Best class per detection
    class_ids = probs.argmax(axis=-1)
    confidences = probs.max(axis=-1)
    
    # Filter by confidence
    mask = confidences > conf_threshold
    filtered_boxes = boxes[mask]
    filtered_scores = confidences[mask]
    filtered_classes = class_ids[mask]
    
    # Apply NMS (use cv2.dnn.NMSBoxes or torchvision.ops.nms)
    # ...
    
    return filtered_boxes, filtered_scores, filtered_classes
```

### Risorse Utili

- **RF-DETR Docs**: https://github.com/roboflow/rf-detr
- **ONNX Runtime**: https://onnxruntime.ai/
- **Roboflow**: https://roboflow.com/

---

**Creato per RaceTagger**  
*Ultimo aggiornamento: 2026-01-18*  
*Testato con: PyTorch 2.8.0, ONNX 1.19.0, rfdetr 1.3.0*

## 🔧 Troubleshooting

### Errore: `torch.export` fails

**Causa**: PyTorch 2.9+ usa il nuovo dynamo exporter di default.

**Soluzione**: Il parametro `dynamo=False` è già incluso nel codice sopra.

### Errore: Position embeddings size mismatch

**Causa**: Stai usando il model class sbagliato (risoluzione diversa dal training).

**Soluzione**: Verifica la risoluzione nel checkpoint e modifica `MODEL_CLASS` e `RESOLUTION`.

### Errore: MPS tensor allocation (Mac M-series)

**Causa**: Metal Performance Shaders ha problemi con torch.export.

**Soluzione**: Il codice usa già `.cpu()` e `PYTORCH_ENABLE_MPS_FALLBACK=1`.

### Errore: `antialias` parameter not supported

**Causa**: Vecchie versioni di rfdetr.

**Soluzione**: Upgrade a rfdetr 1.3.0 (già fatto nell'installazione).

### Modello troppo grande per Colab

**Soluzione**: Usa Colab Pro per più RAM/GPU, oppure esegui localmente.